# Recover Boundary velocity (Least Square Sense) 
> jeonghyeop kim (2021/7/27) <font color=red>(still working)</font>
> \
> My first inversion code written in python

This code uses:
1. data vector: **`vel_on_boundary_wrt_URcorner.gmt`** & \
2. G-matrix components: **`vel_BC_%d_%d.gmt`**, (% 001..024; 100, 010, 001) \
          100 : x-rotation; 010 : y-rotation; 001 : z-rotation
3. For **`vel_BC_%d_%d.gmt`**, I don't know how Ali generates them yet. Talk to him or Bill



In [6]:
#import numpy and pandas
import numpy as np
import pandas as pd

### `STEP1`: **BUILD a data vector,  $\vec{d}$**

In [7]:
# load input file
inputFILE = "vel_on_boundary_wrt_URcorner.gmt" # input file name
df_input = pd.read_csv(inputFILE, header = None, sep =' ')
df_input.columns = ['lon','lat','ve','vn','se','sn','corr']

In [8]:
# BUILD a 'data' vector d along with coordinate information

df_data_x = df_input.iloc[:,[0,1,2]]  # saved vx data on the boudnary
df_data_y = df_input.iloc[:,[0,1,3]]  # saved vn data on the boundary

df_data_x=df_data_x.rename(columns ={'ve': 'velo'}) #column name change
df_data_y=df_data_y.rename(columns ={'vn': 'velo'}) #column name change

df_data_x['flag']=np.array([1] * len(df_data_x)) # 1 = velocity east-component
df_data_y['flag']=np.array([2] * len(df_data_y)) # 2 = velocity north-component

# !! SORT_VALUES !! # lat ascending first, and then lon ascending.
# This step is very important to build the G matrix, G, which
# has rows correspoding to the rows of the data vector, d, that have
# the same coordinates!

df_data_x.sort_values(['lat', 'lon'], ascending=[True, True])
df_data_y.sort_values(['lat', 'lon'], ascending=[True, True])


# MERGE two columns (n*1) into a new column (2n*1)
# > ignore_index = True : 
# >   have one continuous index numbers,
# >     ignorning each of the two dfs original indices

frames=[df_data_x,df_data_y]
df_data=pd.concat(frames,ignore_index=True) # merge the two dataFrames into one



#df_data #to check the data vector

NOTE:   
>If your data have errors, \
build a **W** matrix in which its diagonal elements \
are of a vector made of 1/errors (in order).  \
Your new data vector **d<sub>w</sub>** = **W** **d**

### `STEP2`: **BUILD G-Matrix, $\bar{\bar{G}}$**

> Comment on the read_csv(sep option) below: \
> **`'(?:,|\s+)'`** - is a RegEx for selecting either comma or any number of consecutive spaces/tabs \
> See the answer at StackOverFlow [click here](https://stackoverflow.com/questions/43784422/pandas-error-when-reading-csv-file-using-sep-and-comment-arguments)

In [9]:
df_G = pd.DataFrame(index = range(len(df_data))) 
# Make a blank G matrix


print('How many files do you have for the boudnary basis functions? :')
HowMany = input()
HowMany = int(HowMany)

for i in range(1,HowMany+1): # i in range(0+1,how many files+1?); i in {001 002 003 ... 999} 

    inputfile_xrot = "vel_BC_"+str(f"{i:03}")+"_100.gmt" # x-rot : 100
    inputfile_yrot = "vel_BC_"+str(f"{i:03}")+"_010.gmt" # y-rot : 010
    inputfile_zrot = "vel_BC_"+str(f"{i:03}")+"_001.gmt" # z-rot : 001

# READ files in order {xrot1, yrot1, zrot1, ..., xrotHowMany, yrotHowMany, zrotHowMany}

    df_xrot=pd.read_csv(inputfile_xrot ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_yrot=pd.read_csv(inputfile_yrot ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_zrot=pd.read_csv(inputfile_zrot ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')

# CHANGE the column names 

    df_xrot.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_yrot.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_zrot.columns = ['lon','lat','ve','vn','se','sn','corr']
    
# BUILD a column vector Gx (i)

    df_xrot_x = df_xrot.iloc[:,[0,1,2]]  # saved vx basis function on the boudnary
    df_xrot_y = df_xrot.iloc[:,[0,1,3]]  # saved vn basis function on the boundary

    df_xrot_x=df_xrot_x.rename(columns ={'ve': 'velo'}) #column name change
    df_xrot_y=df_xrot_y.rename(columns ={'vn': 'velo'}) #column name change

    df_xrot_x['flag']=np.array([1] * len(df_xrot_x)) # 1 = velocity east-component
    df_xrot_y['flag']=np.array([2] * len(df_xrot_y)) # 2 = velocity north-component
    
    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_xrot_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_xrot_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices   
    frames_Gx=[df_xrot_x,df_xrot_y]
    df_Gx=pd.concat(frames_Gx,ignore_index=True) # merge the two dataFrames into one

    
    
# BUILD a column vector Gy (i)

    df_yrot_x = df_yrot.iloc[:,[0,1,2]]  # saved vx basis function on the boudnary
    df_yrot_y = df_yrot.iloc[:,[0,1,3]]  # saved vn basis function on the boundary

    df_yrot_x=df_yrot_x.rename(columns ={'ve': 'velo'}) #column name change
    df_yrot_y=df_yrot_y.rename(columns ={'vn': 'velo'}) #column name change

    df_yrot_x['flag']=np.array([1] * len(df_yrot_x)) # 1 = velocity east-component
    df_yrot_y['flag']=np.array([2] * len(df_yrot_y)) # 2 = velocity north-component

    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_yrot_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_yrot_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gy=[df_yrot_x,df_yrot_y]
    df_Gy=pd.concat(frames_Gy,ignore_index=True) # merge the two dataFrames into one
    
    
# BUILD a column vector Gz (i)

    df_zrot_x = df_zrot.iloc[:,[0,1,2]]  # saved vx basis function on the boudnary
    df_zrot_y = df_zrot.iloc[:,[0,1,3]]  # saved vn basis function on the boundary

    df_zrot_x=df_zrot_x.rename(columns ={'ve': 'velo'}) #column name change
    df_zrot_y=df_zrot_y.rename(columns ={'vn': 'velo'}) #column name change

    df_zrot_x['flag']=np.array([1] * len(df_zrot_x)) # 1 = velocity east-component
    df_zrot_y['flag']=np.array([2] * len(df_zrot_y)) # 2 = velocity north-component
   
    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_zrot_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_zrot_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gz=[df_zrot_x,df_zrot_y]
    df_Gz=pd.concat(frames_Gz,ignore_index=True) # merge the two dataFrames into one
    
    
# SAVE G-matrix
# Gmatrix = [Gxrot(1) Gyrot(1) Gzrot(1) ... Gxrot(HowMany) Gyrot(HowMany) Gzrot(HowMany)]
    
    df_G["G_x"+str(i)] = df_Gx.loc[:,['velo']]
    df_G["G_y"+str(i)] = df_Gy.loc[:,['velo']]
    df_G["G_z"+str(i)] = df_Gz.loc[:,['velo']]

How many files do you have for the boudnary basis functions? :


 24


In [14]:
#Check the G-matrix
# outputFILE='Gmatrix_test.dat'
# df_G.to_csv(outputFILE, header=None, index=None, sep=' ',float_format='%f6.3')
df_G

,G_x1,G_y1,G_z1,G_x2,G_y2,G_z2,G_x3,G_y3,G_z3,G_x4,...,G_z21,G_x22,G_y22,G_z22,G_x23,G_y23,G_z23,G_x24,G_y24,G_z24
0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.000,0.000,0.0,-0.000,0.000,0.0,-0.000,0.000,0.0
1,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.000,0.000,0.0,-0.000,0.000,0.0,-0.000,0.000,0.0
2,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.000,0.000,0.0,-0.000,0.000,0.0,-0.000,0.000,0.0
3,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.000,0.000,0.0,-0.000,0.000,0.0,-0.000,0.000,0.0
4,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0,-0.000,0.000,0.0,-0.000,0.000,0.0,-0.000,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,...,-0.0,-5.698,2.851,0.0,-0.000,0.000,0.0,0.000,-0.000,-0.0
92,-0.0,0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,-0.0,...,-0.0,-3.205,1.603,-0.0,-3.205,1.603,0.0,0.356,-0.178,0.0
93,-0.0,0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,-0.0,...,-0.0,-0.000,0.000,-0.0,-5.699,2.849,0.0,0.000,-0.000,0.0
94,-0.0,0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,-0.0,...,0.0,0.417,-0.209,-0.0,-4.146,2.072,-0.0,-2.221,1.110,0.0


In [15]:
# INVERSION HERE! pd.matrix()?